# Smart-G demo notebook

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html).

**Table of contents**

* [How to run Smart-G](#run)
* [How to view the results](#view)
* [A few examples](#examples)
* [Difference between simulations](#difference)
* [Looping over parameters](#loop)
* [Interactive simulation](#interactive)

In [ ]:
%pylab inline
from smartg import Smartg, RoughSurface, LambSurface, FlatSurface, Environment
from PAR import SpecInt, SpecInt2, Irr, ReadREPTRAN_bands
from profile.profil import Profile, AeroOPAC, CloudOPAC
from luts import LUT, MLUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from water.iop_spm import IOP_SPM
from water.iop_mm import IOP_MM
from smartg_view import semi_polar, smartg_view
from IPython.html.widgets.interaction import interact, interact_manual

<a id="run"></a>
## How to run Smart-G

In [ ]:
# run SMART-G, and skip if file exists
# this simulation uses an AFGL tropical atmosphere without aerosols
job = Smartg('SMART-G-PP', THVDEG=30, wl=500., NBPHOTONS=1e9,
                    atm=Profile('afglt'),
                    surf=RoughSurface(SUR=1), overwrite=True)
print 'done! the result hdf file is', job.output

<a id="view"></a>
## How to view the results

In [ ]:
# basic visualization
job.view()

In [ ]:
M = job.read()  # read all datasets
M.print_info()  # show all datasets with their dimensions

In [ ]:
# show dataset 'Q_up (TOA)' with a transect at phi=45 (optional)
# also specify a color scale
semi_polar(M['Q_up (TOA)'], index=Idx(45), vmin=-0.15, vmax=0.05)

In [ ]:
# more advanced visualization:
# two plots side by side, including transects, using a larger figure
fig = figure(figsize=(12, 6))
I_up = job.read('I_up (TOA)')
semi_polar(I_up, index=Idx(5.), rect=221, sub=223, swap=False, fig=fig)
semi_polar(I_up, index=Idx(30.), rect=222, sub=224, swap=True, fig=fig)  # reversed axes

In [ ]:
# interactive display
interact_manual(lambda vmax, phi:
                semi_polar(lut=I_up, index=Idx(phi), vmin=0, vmax=vmax),
                vmax=(0,1.,0.01),
                phi=(0, 180),
               )

<a id="examples"></a>
## A few examples

In [ ]:
# Basic Rayleigh example, no surface
Smartg('SMART-G-PP', wl=500., THVDEG=30.,
       NBPHOTONS=1e9, atm=Profile('afglt'), overwrite=True).view()

In [ ]:
# only Rayleigh with a custom grid
Smartg('SMART-G-PP', wl=500., DEPO=0., NBPHOTONS=1e9,
        atm=Profile('afglt', grid='100[75]25[5]10[1]0'), overwrite=True).view()

In [ ]:
# Rayleigh + aerosols
aer = AeroOPAC('urban', 0.4, 550.)
pro = Profile('afglms', aer=aer)
Smartg('SMART-G-PP', wl=300., THVDEG=60, atm=pro, NBPHOTONS=1e8, overwrite=True).view()

In [ ]:
# atmosphere + surface
Smartg('SMART-G-PP', 490., NBPHOTONS=1e7, THVDEG=45.,
        atm=Profile('afglms'),
        surf=LambSurface(ALB=0.1),
        overwrite=True).view()

In [ ]:
# atmosphere + surface + océan
Smartg('SMART-G-PP', 490., NBPHOTONS=1e7, THVDEG=30.,
            atm=Profile('afglms'),
            surf=RoughSurface(),
            water=IOP_MM(1.), overwrite=True).view()

In [ ]:
# surface + océan (SPM model)
Smartg('SMART-G-PP', 750., THVDEG=30., NBPHOTONS=1e9,
        surf=RoughSurface(),
        water=IOP_SPM(SPM=100.), overwrite=True).view()

In [ ]:
# océan seul (chl model)
jj=Smartg('SMART-G-PP', wl=750., THVDEG=30.,
        water=IOP_SPM(100.), NBPHOTONS=1e9, overwrite=True)
jj.view()
M=jj.read()
M.print_info()

In [ ]:
semi_polar(jj.read()['I_up (TOA)'].sub[0,:,:])

In [ ]:
# multispectral
pro = Profile('afglt',
              grid=[100, 75, 50, 30, 20, 10, 5, 1, 0.],  # optional, otherwise use default grid
              pfgrid=[100, 20, 0.],   # optional, otherwise use a single band 100-0
              pfwav=[400, 500, 600], # optional, otherwise phase functions are calculated at all bands
              aer=AeroOPAC('maritime_clean', 0.3, 550.),
              verbose=True)
job = Smartg('SMART-G-PP', wl = np.linspace(400, 600, 10.),
             THVDEG=60.,
             atm=pro,
             surf=RoughSurface(),
             water=IOP_SPM(1.),
             overwrite=True)
semi_polar(job.read()['I_up (TOA)'].sub[0,:,:])

In [ ]:
# REPTRAN (example 1)
repname = 'reptran_solar_msg' # k distribution file, here MSG/SEVIRI solar channels
BAND = 'msg1_seviri_ch006'
surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34)
atm=Profile('afglmw')  
# Read Reptran band characteristics
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname,BAND=BAND,FULL=True)
job = Smartg('SMART-G-SP', THVDEG=30, wl=bands, NBPHOTONS=1e8, atm=atm, surf=surf, overwrite=True)

In [ ]:
M=job.read()
MM_l=[]
for k in range(len(M.luts)):
    MM_tmp = M.luts[k]
    MM=LUT(MM_tmp.data,axes=[wi.data,MM_tmp.axes[1],MM_tmp.axes[2]],desc=MM_tmp.desc,
           names=['Wavelengths',MM_tmp.names[1],MM_tmp.names[2]],attrs=MM_tmp.attrs)
    MM_l.append(MM)
M2 = MLUT(MM_l)
print '--M-----------',M2['I_up (TOA)'].print_info()
fig=figure(figsize=(8,4))
semi_polar(M2['I_up (TOA)'].sub[0,:,:],rect=121,fig=fig,vmin=0,vmax=0.2)
semi_polar(M2['I_up (TOA)'].sub[1,:,:],rect=122,fig=fig,vmin=0,vmax=0.2)
s = SpecInt2(wi,wb,ex,we,dl,M=M2,field='I_up (TOA)')
print '--S-----------',s.print_info()
fig2=figure(figsize=(4,4))
semi_polar(s.sub[0,:,:],vmin=0,vmax=0.2,rect=111,fig=fig2)

In [ ]:
# REPTRAN (example 2)
repname = 'reptran_solar_medium' # k distribution file, here full solar channels at mdeium resolution
LMIN=765. # nm within O2A bands
LMAX=768. # nm
SAMPLING=1 # sampling of individual channels 1: all microchannels
surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34)
atm=Profile('afglmw')  
# Read Reptran band characteristics
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname, LMIN=LMIN, LMAX=LMAX, SAMPLING=SAMPLING, FULL=True)
job = Smartg('SMART-G-PP', THVDEG=30, wl=bands, NBPHOTONS=1e9, atm=atm, surf=surf, overwrite=True)

In [ ]:
M=job.read()
MM_l=[]
for k in range(len(M.luts)):
    MM_tmp = M.luts[k]
    MM=LUT(MM_tmp.data,axes=[wi.data,MM_tmp.axes[1],MM_tmp.axes[2]],desc=MM_tmp.desc,
           names=['Wavelengths',MM_tmp.names[1],MM_tmp.names[2]],attrs=MM_tmp.attrs)
    MM_l.append(MM)
M2 = MLUT(MM_l)
print '--M-----------',M2['I_up (TOA)'].print_info()
fig=figure(figsize=(8,4))
semi_polar(M2['I_up (TOA)'].sub[Idx(765.5),:,:],rect=121,fig=fig,vmin=0,vmax=0.1)
semi_polar(M2['I_up (TOA)'].sub[Idx(767.5),:,:],rect=122,fig=fig,vmin=0,vmax=0.1)
s = SpecInt2(wi,wb,ex,we,dl,M=M2,field='I_up (TOA)')
print '--S-----------',s.print_info()
fig2=figure(figsize=(8,4))
semi_polar(s.sub[Idx(765.5),:,:],vmin=0,vmax=0.1,sub=223,index=Idx(140),rect=221,fig=fig2)
semi_polar(s.sub[:,Idx(140),:], rect=222, sub=224, index=Idx(30),swap=True,fig=fig2,vmin=0,vmax=0.1)

<a id="loop"></a>
## Looping over parameters

In [ ]:
# loop over some wavelengths
MLUTS = []
for wl in linspace(400, 800, 10):
    job = Smartg('SMART-G-PP', THVDEG=30,
                    wl=wl, NBPHOTONS=1e7,
                    atm=Profile('afglt'),
                    surf=RoughSurface(), overwrite=True)
    print job.output

    MLUTS.append(job.read())

M = merge(MLUTS, ['LAMBDA'])
M['I_up (TOA)'].print_info()
semi_polar(M['I_up (TOA)'].sub[:,0,:], swap=True, index=12)

<a id="difference"></a>
## Difference between two LUTs

In [ ]:
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
jobs = []
for ex in ['SMART-G-PP', 'SMART-G-SP']:
    jobs.append(Smartg(ex, THVDEG=60, wl=443., NBPHOTONS=1e10,
                         NBTHETA=20., NBPHI=20.,
                        atm=Profile('afglt'), overwrite=True))
    
PP = jobs[0].read('I_up (TOA)')
SP = jobs[1].read('I_up (TOA)')

# show relative difference
semi_polar(100.*(SP-PP)/PP, Idx(175.), fig=figure(figsize(8, 8)), vmin=-2, vmax=2)

In [ ]:
semi_polar(SP/PP, Idx(90.), fig=figure(figsize(8, 8)),vmin=0.97,vmax=1.03)

<a id="interactive"></a>
## Interactive simulation

In [ ]:
from IPython.html.widgets.interaction import interact, interact_manual

def simulate(thvdeg, surface, aerosol_model, aot550, wavelength):
    surf = {True:RoughSurface(), False:None}[surface]
    aer = AeroOPAC(aerosol_model, aot550, 550.)
    Smartg('SMART-G-SP', THVDEG=thvdeg,
             wl=wavelength, NBPHOTONS=1e8,
             NBTHETA=90., NBPHI=90.,
             atm=Profile('afglt', aer=aer), surf=surf, OUTPUT_LAYERS=3,
             overwrite=True).view(QU=True,field='up (0+)')

interact_manual(simulate, thvdeg=(0, 90), surface=True,
                aerosol_model=AeroOPAC.listStandardAerosolFiles(), aot550=(0.001, 5., 0.01),
                wavelength=(400., 800.))